In [1]:
!pip install openai
!pip install llama-index
!pip install trulens-eval
!pip install llama-index-llms-openai
!pip install llama-index-embeddings-openai
!pip install pypdf==3.17.2

In [3]:
import os
import openai
import io
import pandas as pd
import llama_index

In [4]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
openai.api_key = os.environ["OPENAI_API_KEY"]

## Read context document

In [5]:
# Initialize models for LlamaIndex
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
embed_model = OpenAIEmbedding()

### Load data, split into chunks

In [44]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader(
    input_files=["Marian_College_Employee_Handbook.pdf","marian_college_roles_and_responsibilities.pdf"]
).load_data()

print(len(documents))

77


In [26]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader(
    input_files=["PC_Troubleshooting_Manuals.pdf"]
).load_data()

print(len(documents))

26


## Index-building functions

In [39]:
from llama_index.core import ServiceContext, VectorStoreIndex
from llama_index.core.node_parser import SentenceWindowNodeParser

def build_basic_index(documents, llm, embed_model=OpenAIEmbedding()):
    basic_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)
    basic_index = VectorStoreIndex.from_documents(
        documents=documents, service_context=basic_context
    )
    return basic_index

def build_sentence_window_index(documents, llm, embed_model=OpenAIEmbedding()):
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=3,
        window_metadata_key="window",
        original_text_metadata_key="original-text"
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser
    )
    sentence_index = VectorStoreIndex.from_documents(
        documents=documents, service_context=sentence_context
    )
    return sentence_index

### Configure TruLens Evaluation framework

In [28]:
from trulens_eval.feedback import Groundedness,GroundTruthAgreement
from trulens_eval import Tru
from trulens_eval import OpenAI, Feedback, TruLlama
import numpy as np


In [45]:
# Initialize the main entry-point to TruLens, Tru
tru = Tru()

openai_provider = OpenAI(model_engine="gpt-3.5-turbo-1106")
#context = App.select_context(query_engine)
qa_df = pd.read_csv("Book2.csv")
qa_set = [{"query": item["Question"], "response": item["Answer"]} for index, item in qa_df.iterrows()]
print(qa_set)

[{'query': 'What is the transport facility for Marian engineering college staff?', 'response': 'The transport facility for Marian Engineering College staff includes arrangements for transport from the college to the city and vice versa. The transport manager is responsible for arranging and maintaining the buses, checking the log books maintained by the drivers, and managing the time of the buses.'}, {'query': 'What is the exit policy for Marian engineering college staff?', 'response': 'The exit policy for Marian Engineering College staff states that the age of superannuation for teaching, technical, and non-teaching staff is as per Kerala Government norm. In case of shortage of experienced faculty, the management can reappoint the faculty on a contract basis for a period of one year, renewable at the discretion of the management. The management has the right to terminate the service of a staff member by giving a notice of 3 months or 3 months pay in lieu of notice if their performance

In [16]:
# Initialize metrics to collect
# Answer relevance
f_qa_relevance = Feedback(
    openai_provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()

# Context relevance
f_qs_relevance = Feedback(
    openai_provider.relevance_with_cot_reasons, name="Context Relevance"
).on_input().on(TruLlama.select_source_nodes().node.text).aggregate(np.mean)

# Groundedness or faithfulness (if model is hallucinating)
grounded = Groundedness(groundedness_provider=openai_provider)
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(TruLlama.select_source_nodes().node.text)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Ground truth agreement or anwer correctness (how correct are the answer)
f_groundtruth = Feedback(
    GroundTruthAgreement(qa_set).agreement_measure, name="Answer Correctness"
).on_input_output()

metrics = [f_qa_relevance, f_qs_relevance, f_groundedness, f_groundtruth]

def get_trulens_recorder(query_engine, app_id):
    tru_recorder = TruLlama(
        query_engine,
        feedbacks=metrics,
        app_id=app_id
    )
    return tru_recorder

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Correctness, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Correctness, input response will be set to __record__.main_output or `Select.RecordOutput` .


## Run Evals for Basic Index

In [30]:
basic_query_index = build_basic_index(documents=documents, llm=llm, embed_model=embed_model)
query_engine = basic_query_index.as_query_engine()

<ipython-input-27-17c77ad79d8e>:4: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  basic_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)


In [31]:
tru_recorder = TruLlama(
        query_engine,
        feedbacks=metrics,
        app_id="LlamaIndex_App2")

In [32]:
basic_query_index = build_basic_index(documents=documents, llm=llm, embed_model=embed_model)
basic_query_engine = basic_query_index.as_query_engine()
basic_recorder = get_trulens_recorder(basic_query_engine, app_id="Basic Query Engine Technical")


<ipython-input-27-17c77ad79d8e>:4: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  basic_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)


In [35]:
with basic_recorder as recording:
    for q in qa_set:
        basic_query_engine.query(q['query'])

Groundedness per statement in source:   0%|          | 0/10 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/5 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/14 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/5 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

In [36]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
# records.head(5)
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Submit this IP Address: 34.106.23.213



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Run Evals for Sentence Window Index

In [46]:
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor
sentence_window_index = build_sentence_window_index(documents=documents, llm=llm, embed_model=embed_model)

postproc = MetadataReplacementPostProcessor(target_metadata_key="window")

sentence_query_engine = sentence_window_index.as_query_engine(
    node_postprocessors=[postproc]
)

sentence_recorder = get_trulens_recorder(
    sentence_query_engine, app_id="Sentence Window Query Engine HR"
)

<ipython-input-39-8ca5bfda2bbe>:17: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(


In [47]:
with sentence_recorder as recording:
    for q in qa_set:
        sentence_query_engine.query(q['query'])

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

...

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]